## Jupyterhub on Kubernetes

This notebook can be used to launch a standard instance of Jupyterhub which includes a proxy server, and an instance of Jupyterhub. 

Notes on the Server: 
`/etc/jupyterhub/config` is where the configuration is for jupyterhub. 



In [2]:
import sys 
import importlib
import ruamel.yaml
config_file='config/config.yaml'  #The configuration value set above.
path='/home/jovyan/work/admin-tools/' #Update this value if running locally.
sys.path.append(path+"lib/kuberutils") #Adds a local library path.
import kuberutils as ku   #This imports some utilities.
importlib.reload(ku)      #This just reloads the utilities.
cf=ku.initialize(path,config_file)  #This initializes a configuration object.

### Print Configuration
Optionally you can print the configuration and common commands for your desired cluster. You can use this as a reference and copy and paste into the terminal.

In [3]:
#This will print common commands for your clustl
print(ruamel.yaml.dump(cf, sys.stdout, Dumper=ruamel.yaml.RoundTripDumper))

#Where are you running notebooks? This is used for path.
docker: true     #True if executing on the Kuberlytics Docker Stack.
docker_path: /home/jovyan/work/admin-tools      #This should not change.
local_path: /Users/jasonkuruzovich/githubdesktop/0_class/admin-tools #This is the local path
cloud_provider: azure   #google or azure
#Start of Google Specific Configuration
g_project: kuberlytics             #Google project name
g_cluster_name: kuberlytics        #Name of your Google Cluster
g_region: us-east1                 #Selection from gcloud compute regions list.
g_zone: us-east1-b                  #Selection from gcloud compute regions list.
g_machine_type: n1-highmem-4        #Type of Server
g_account: jkuruzovich@gmail.com    #Email Associated with the account.
g_authorization_file: auth.json     #Service account authorization file.
g_service_account_name: kuberlytics2  #Service account name.
g_fixedip_namespace: jupyterhub-dojo
g_num_nodes: 1                      #The default nu

### Set Jupyterhub Config
This will setup a basic configuration file for installing Jupyterhub. 

In [4]:
ku.set_jupyterhub_config(cf)

Executing rm -rf /home/jovyan/work/admin-tools/config/jupyterhub/dojo:
 rm -rf /home/jovyan/work/admin-tools/config/jupyterhub/dojo
Executing mkdir /home/jovyan/work/admin-tools/config/jupyterhub/dojo:
 mkdir /home/jovyan/work/admin-tools/config/jupyterhub/dojo
Executing openssl rand -hex 32 > /home/jovyan/work/admin-tools/config/jupyterhub/dojo/cookie_secret.txt:
 openssl rand -hex 32 > /home/jovyan/work/admin-tools/config/jupyterhub/dojo/cookie_secret.txt
Executing openssl rand -hex 32 > /home/jovyan/work/admin-tools/config/jupyterhub/dojo/secret_token.txt:
 openssl rand -hex 32 > /home/jovyan/work/admin-tools/config/jupyterhub/dojo/secret_token.txt
Executing wget -P /home/jovyan/work/admin-tools/config/jupyterhub/dojo https://raw.githubusercontent.com/jupyterhub/zero-to-jupyterhub-k8s/master/jupyterhub/values.yaml:
 wget -P /home/jovyan/work/admin-tools/config/jupyterhub/dojo https://raw.githubusercontent.com/jupyterhub/zero-to-jupyterhub-k8s/master/jupyterhub/values.yaml
Executing 

## PLEASE TAKE A MINUTE TO UPDATE YOUR CONFIG FILE.

If you will be keeping this instance for a while, update your config/config.yaml file so that you don't. 

```
#Start of Azure Configuration
jup_rebuild_config: false

```



## Download Helm Repo

In [5]:
#Add the jupyterhub repo.
print(ku.bash_command('jup_repo',cf))

Executing jup_repo:
 helm repo add jupyterhub https://jupyterhub.github.io/helm-chart/ && helm repo update
"jupyterhub" has been added to your repositories
Hang tight while we grab the latest from your chart repositories...
...Skip local chart repository
...Successfully got an update from the "jupyterhub" chart repository
...Successfully got an update from the "stable" chart repository
Update Complete. ⎈ Happy Helming!⎈ 



In [16]:
#This will install the jupyterhub instance.
print(ku.bash_command('jup_install',cf))

Executing jup_install:
 helm install jupyterhub/jupyterhub --version=v0.5.0-e154ab7 --name=dojo --namespace=dojo -f /home/jovyan/work/admin-tools/config/jupyterhub/dojo/config.yaml
NAME:   dojo
LAST DEPLOYED: Sat Nov  4 02:43:12 2017
NAMESPACE: dojo
STATUS: DEPLOYED

RESOURCES:
==> v1/Secret
NAME        TYPE    DATA  AGE
hub-secret  Opaque  2     11s

==> v1/ConfigMap
NAME        DATA  AGE
hub-config  21    11s

==> v1/PersistentVolumeClaim
NAME        STATUS   VOLUME   CAPACITY  ACCESSMODES  STORAGECLASS  AGE
hub-db-dir  Pending  default  11s

==> v1/Service
NAME          CLUSTER-IP    EXTERNAL-IP  PORT(S)                     AGE
hub           10.0.187.205  <none>       8081/TCP                    11s
proxy-public  10.0.218.111  <pending>    80:32374/TCP,443:31489/TCP  10s
proxy-api     10.0.171.219  <none>       8001/TCP                    10s
proxy-http    10.0.239.229  <none>       8000/TCP                    10s

==> v1beta1/Deployment
NAME            DESIRED  CURRENT  UP-TO-DATE 

In [17]:
#This will check on the status of Jupyterhub.
print(ku.bash_command('jup_describe', cf))

Executing jup_describe:
 kubectl --namespace=dojo get pod
NAME                              READY     STATUS    RESTARTS   AGE
hub-deployment-2807160951-prbf0   0/1       Pending   0          36s
proxy-2306483264-b98mj            2/2       Running   0          36s



In [21]:
#Get the ip/url. 
#cf=ku.get_jupyterhub_ip(cf)
ku.bash_command('jup_get_ip',cf)

Executing jup_get_ip:
 kubectl --namespace=dojo get svc proxy-public


'NAME           TYPE           CLUSTER-IP     EXTERNAL-IP     PORT(S)                      AGE\nproxy-public   LoadBalancer   10.0.218.111   40.123.45.141   80:32374/TCP,443:31489/TCP   20m\n'

## Update Authorization
When just starting out, the default configuration launches with no password.  While there is still a username and login screen, 


In [11]:
#This is how we set the authororization
#ku.update_config('auth','jup_github_auth', cf)
#ku.update_config('auth','jup_google_auth', cf)
ku.update_config('auth','jup_dummy_auth', cf)


In [12]:
#Add Administrators
ku.update_config('admin','jup_admin', cf)

### Update the Singleuser Container
This can allow you to utilize a different container. 


In [13]:
ku.update_config('singleuser','jup_singleuser', cf)

In [ ]:
#Upgrading Jupyterhub 
print(ku.bash_command('jup_upgrade',cf))

Executing jup_upgrade:
 helm upgrade dojo jupyterhub/jupyterhub --version=v0.5.0-e154ab7 -f /home/jovyan/work/admin-tools/config/jupyterhub/dojo/config.yaml


Adding SSL Support via Let's Encrypt
helm repo update 
helm install --name=letsencrypt --namespace=kube-system stable/kube-lego \
             --set config.LEGO_EMAIL=<your-email> \
             --set config.LEGO_URL=https://acme-v01.api.letsencrypt.org/directory

## Cleanup the Installation
This will cleanup the installation, deleting the instance of Jupyterhub.

In [12]:
#Delete Jupyterhub with Helm.
print(ku.bash_command('jup_delete_instance',cf))

Executing jup_delete_instance:
 helm delete dojo --purge
release "dojo" deleted



In [14]:
#Delete Jupyterhub with Helm.
print(ku.bash_command('jup_delete_namespace',cf))

Executing jup_delete_namespace:
 kubectl delete namespace dojo
Error from server (Conflict): Operation cannot be fulfilled on namespaces "dojo": The system is ensuring all content is removed from this namespace.  Upon completion, this namespace will automatically be purged by the system.

